In [3]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing sklearn libraries 
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, mean_absolute_error

# importing models
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
# Loading Data sets
data = pd.read_csv("C:\\Users\\123\\Desktop\\python project for data analitycs\\Data\\customer_behavior_dataset.xls")

In [5]:
data.head()

,age,annual_income,monthly_spend,avg_session_time,visits_per_month,items_per_purchase,discount_used_pct,days_since_last_purchase,cart_abandon_rate,customer_tenure_months,gender,region,device_type,payment_method,membership_type,customer_segment
0,56.0,1.436647e+06,8168.887229,2.018793,10.0,1.0,27.533169,48.0,0.345017,69.0,Male,NaN,Mobile,Credit Card,Platinum,Medium_Value_Customer
1,69.0,1.109740e+06,22375.492576,NaN,15.0,7.0,20.160388,8.0,0.054733,98.0,NaN,West,Mobile,Credit Card,Silver,High_Value_Customer
2,NaN,3.441890e+05,8354.207778,11.683007,NaN,2.0,41.682201,57.0,NaN,106.0,Female,East,Mobile,NaN,Silver,Medium_Value_Customer
3,32.0,6.547298e+05,33426.674454,4.371077,13.0,NaN,40.733550,44.0,0.418695,1.0,Female,NaN,Mobile,UPI,Gold,High_Value_Customer
4,NaN,1.180073e+06,18453.225306,13.017717,NaN,7.0,30.591551,21.0,NaN,39.0,Female,North,Tablet,Net Banking,Platinum,Medium_Value_Customer


In [17]:
df = data.copy()

In [18]:
df.isnull().sum()

age                         50
annual_income               50
monthly_spend               50
avg_session_time            50
visits_per_month            50
items_per_purchase          50
discount_used_pct           50
days_since_last_purchase    50
cart_abandon_rate           50
customer_tenure_months      50
gender                      50
region                      50
device_type                 50
payment_method              50
membership_type             50
customer_segment             0
dtype: int64

In [8]:
df.size

8000

In [12]:
def simplyfy_data(
    data,
    target_col,
    col_threshold=50,
    row_threshold=5,
    num_strategy="median",
    cat_strategy="mode"
):  
    # Loading required libraries
    import pandas as pd
    import numpy as np
    """
    Handling duplicates value.
    Handles missing values using row-wise and column-wise strategies.
    Handling Outliers

    """

    # Step 1: Create a copy to avoid modifying original data
    df = data.copy()

    #Droping duplicated values
    df.drop_duplicates()

    # Step 2: Separate features and target
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Step 3: Handle missing values row-wise
    # Drop rows where missing percentage > row_threshold
    row_missing_pct = df.isnull().mean(axis=1) * 100
    df = df[row_missing_pct <= row_threshold]

    # Step 4: Calculate column-wise missing percentage
    col_missing_pct = df.isnull().mean() * 100

    # Step 5: Drop columns with missing % > col_threshold
    drop_cols = col_missing_pct[col_missing_pct > col_threshold].index
    df.drop(columns=drop_cols, inplace=True)

    # Recalculate after dropping
    col_missing_pct = df.isnull().mean() * 100

    # Step 6: Handle remaining columns
    for col in df.columns:

        # Skip target column
        if col == target_col:
            continue

        # Identify column type
        is_numeric = pd.api.types.is_numeric_dtype(df[col])

        # Case 1: Missing < 5% → drop rows
        if col_missing_pct[col] < 5:
            df = df[df[col].notna()]

        # Case 2: Missing between 5% and 30% → simple imputation
        elif 5 <= col_missing_pct[col] <= 30:
            if is_numeric:
                if num_strategy == "mean":
                    df[col].fillna(df[col].mean(), inplace=True)
                else:
                    df[col].fillna(df[col].median(), inplace=True)
            else:
                df[col].fillna(df[col].mode()[0], inplace=True)

        # Case 3: Missing between 30% and 50%
        # Use correlation with target for numeric columns
        elif 30 < col_missing_pct[col] <= 50:

            if is_numeric and pd.api.types.is_numeric_dtype(df[target_col]):
                corr = df[[col, target_col]].corr().iloc[0, 1]

                # Drop weakly correlated columns
                if abs(corr) < 0.1:
                    df.drop(columns=[col], inplace=True)
                else:
                    df[col].fillna(df[col].median(), inplace=True)
            else:
                # Categorical columns → mode imputation
                df[col].fillna(df[col].mode()[0], inplace=True)

    # Handling Outliers using IQR method
    for col in df.select_dtypes(include='number').columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        IQR = q3 - q1

        lower = q1 - 1.5 * IQR
        upper = q3 + 1.5 * IQR

        df = df[(df[col] >= lower) & (df[col] <= upper)]

    return df

In [14]:
df = simplyfy_data(df,"customer_segment")

In [15]:
df.isnull().sum()

age                         0
annual_income               0
monthly_spend               0
avg_session_time            0
visits_per_month            0
items_per_purchase          0
discount_used_pct           0
days_since_last_purchase    0
cart_abandon_rate           0
customer_tenure_months      0
gender                      0
region                      0
device_type                 0
payment_method              0
membership_type             0
customer_segment            0
dtype: int64

In [19]:
df.isnull().sum().sum()*100/df.size

np.float64(9.375)

In [21]:
df.dropna(inplace=True)

In [22]:
df.isnull().sum()

age                         0
annual_income               0
monthly_spend               0
avg_session_time            0
visits_per_month            0
items_per_purchase          0
discount_used_pct           0
days_since_last_purchase    0
cart_abandon_rate           0
customer_tenure_months      0
gender                      0
region                      0
device_type                 0
payment_method              0
membership_type             0
customer_segment            0
dtype: int64

In [25]:
for col in df.columns:
    le = LabelEncoder()
    if type(col) == str:
        df[col] = le.fit_transform(df[col])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 6 to 497
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   age                       101 non-null    int64
 1   annual_income             101 non-null    int64
 2   monthly_spend             101 non-null    int64
 3   avg_session_time          101 non-null    int64
 4   visits_per_month          101 non-null    int64
 5   items_per_purchase        101 non-null    int64
 6   discount_used_pct         101 non-null    int64
 7   days_since_last_purchase  101 non-null    int64
 8   cart_abandon_rate         101 non-null    int64
 9   customer_tenure_months    101 non-null    int64
 10  gender                    101 non-null    int64
 11  region                    101 non-null    int64
 12  device_type               101 non-null    int64
 13  payment_method            101 non-null    int64
 14  membership_type           101 non-null    int64

In [27]:
df.head()

,age,annual_income,monthly_spend,avg_session_time,visits_per_month,items_per_purchase,discount_used_pct,days_since_last_purchase,cart_abandon_rate,customer_tenure_months,gender,region,device_type,payment_method,membership_type,customer_segment
6,17,63,17,21,3,6,23,40,92,55,1,2,0,0,0,2
7,31,77,68,46,8,6,12,57,85,6,1,1,0,0,2,2
12,19,32,84,56,8,1,8,44,69,25,1,2,0,1,2,0
36,39,56,42,9,3,0,42,39,1,16,0,0,1,2,2,2
39,43,100,100,100,14,5,15,57,100,27,0,1,0,3,0,2


In [31]:
df.corr()["customer_segment"]

age                        -0.213523
annual_income              -0.071140
monthly_spend              -0.530817
avg_session_time           -0.015244
visits_per_month           -0.048010
items_per_purchase          0.102254
discount_used_pct          -0.076613
days_since_last_purchase   -0.061758
cart_abandon_rate           0.043388
customer_tenure_months      0.023470
gender                     -0.001826
region                     -0.123121
device_type                 0.028978
payment_method              0.016677
membership_type            -0.101957
customer_segment            1.000000
Name: customer_segment, dtype: float64

In [32]:
x = df.drop("customer_segment",axis = 1)
y = df["customer_segment"]

In [33]:
x.head()

,age,annual_income,monthly_spend,avg_session_time,visits_per_month,items_per_purchase,discount_used_pct,days_since_last_purchase,cart_abandon_rate,customer_tenure_months,gender,region,device_type,payment_method,membership_type
6,17,63,17,21,3,6,23,40,92,55,1,2,0,0,0
7,31,77,68,46,8,6,12,57,85,6,1,1,0,0,2
12,19,32,84,56,8,1,8,44,69,25,1,2,0,1,2
36,39,56,42,9,3,0,42,39,1,16,0,0,1,2,2
39,43,100,100,100,14,5,15,57,100,27,0,1,0,3,0


In [34]:
y.head()

6     2
7     2
12    0
36    2
39    2
Name: customer_segment, dtype: int64

In [35]:
# train_test splitting
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=42)

In [36]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((80, 15), (21, 15), (80,), (21,))

In [37]:
scl = StandardScaler()

In [38]:
x_train_scl = scl.fit_transform(x_train)
x_test_scl = scl.transform(x_test)

In [39]:
Nb = GaussianNB()

In [40]:
Nb.fit(x_train_scl,y_train)

,"priors priors: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None
,"var_smoothing var_smoothing: float, default=1e-9Portion of the largest variance of all features that is added tovariances for calculation stability... versionadded:: 0.20",1e-09


In [43]:
y_pred = Nb.predict(x_test_scl)

In [44]:
accuracy_score(y_test,y_pred)

0.7619047619047619

In [45]:
knn = KNeighborsClassifier(n_neighbors=3)

In [46]:
knn.fit(x_train_scl,y_train)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",3
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [47]:
svc = SVC(kernel='rbf', random_state=42)
svc.fit(x_train_scl, y_train)

,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive. The penaltyis a squared l2 penalty. For an intuitive visualization of the effectsof scaling the regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm. Ifnone is given, 'rbf' will be used. If a callable is given it is used topre-compute the kernel matrix from data matrices; that matrix should bean array of shape ``(n_samples, n_samples)``. For an intuitivevisualization of different kernel types see:ref:`sphx_glr_auto_examples_svm_plot_svm_kernels.py`.",'rbf'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3
,"gamma gamma: {'scale', 'auto'} or float, default='scale'Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.- if ``gamma='scale'`` (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,- if 'auto', uses 1 / n_features- if float, must be non-negative... versionchanged:: 0.22 The default value of ``gamma`` changed from 'auto' to 'scale'.",'scale'
,"coef0 coef0: float, default=0.0Independent term in kernel function.It is only significant in 'poly' and 'sigmoid'.",0.0
,"shrinking shrinking: bool, default=TrueWhether to use the shrinking heuristic.See the :ref:`User Guide `.",True
,"probability probability: bool, default=FalseWhether to enable probability estimates. This must be enabled priorto calling `fit`, will slow down that method as it internally uses5-fold cross-validation, and `predict_proba` may be inconsistent with`predict`. Read more in the :ref:`User Guide `.",False
,"tol tol: float, default=1e-3Tolerance for stopping criterion.",0.001
,"cache_size cache_size: float, default=200Specify the size of the kernel cache (in MB).",200
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to class_weight[i]*C forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",None
,"verbose verbose: bool, default=FalseEnable verbose output. Note that this setting takes advantage of aper-process runtime setting in libsvm that, if enabled, may not workproperly in a multithreaded context.",False


In [48]:
logistic = LogisticRegression()

In [49]:
logistic.fit(x_train_scl,y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [50]:
dt = DecisionTreeClassifier()

In [51]:
dt.fit(x_train,y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curre

In [52]:
rf = RandomForestClassifier(random_state=42)

In [53]:
rf.fit(x_train,y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [54]:
# Base estimator: Decision Tree (stump)
base_estimator = DecisionTreeClassifier(max_depth=1)

In [55]:
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, learning_rate=0.5, random_state=42)

In [57]:
adaboost.fit(x_train,y_train)

,"estimator estimator: object, default=NoneThe base estimator from which the boosted ensemble is built.Support for sample weighting is required, as well as proper``classes_`` and ``n_classes_`` attributes. If ``None``, thenthe base estimator is :class:`~sklearn.tree.DecisionTreeClassifier`initialized with `max_depth=1`... versionadded:: 1.2 `base_estimator` was renamed to `estimator`.",DecisionTreeC...r(max_depth=1)
,"n_estimators n_estimators: int, default=50The maximum number of estimators at which boosting is terminated.In case of perfect fit, the learning procedure is stopped early.Values must be in the range `[1, inf)`.",50
,"learning_rate learning_rate: float, default=1.0Weight applied to each classifier at each boosting iteration. A higherlearning rate increases the contribution of each classifier. There isa trade-off between the `learning_rate` and `n_estimators` parameters.Values must be in the range `(0.0, inf)`.",0.5
,"random_state random_state: int, RandomState instance or None, default=NoneControls the random seed given at each `estimator` at eachboosting iteration.Thus, it is only used when `estimator` exposes a `random_state`.Pass an int for reproducible output across multiple function calls.See :term:`Glossary `.",42
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",1
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None


In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
gbc = GradientBoostingClassifier(n_estimators=300,
                                 learning_rate=0.05,
                                 random_state=100,
                                 max_features=5 )
                                 

In [62]:
gbc.fit(x_train,y_train)

,"loss loss: {'log_loss', 'exponential'}, default='log_loss'The loss function to be optimized. 'log_loss' refers to binomial andmultinomial deviance, the same as used in logistic regression.It is a good choice for classification with probabilistic outputs.For loss 'exponential', gradient boosting recovers the AdaBoost algorithm.",'log_loss'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.For an example of the effects of this parameter and its interaction with``subsample``, see:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_regularization.py`.",0.05
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",300
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",1.0
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are'friedman_mse' for the mean squared error with improvement score byFriedman, 'squared_error' for mean squared error. The default value of'friedman_mse' is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",3
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``,

In [64]:
y_pred_knn = knn.predict(x_test_scl)
y_pred_svc = svc.predict(x_test_scl)
y_pred_log = logistic.predict(x_test_scl)
y_pred_dt = dt.predict(x_test)
y_pred_rf = rf.predict(x_test)
y_pred_adab = adaboost.predict(x_test)
y_pred_gb = gbc.predict(x_test)

In [65]:
print("Accuracy score of KNN Model is      :", accuracy_score(y_test, y_pred_knn))
print("Accuracy score of SVC Model is      :", accuracy_score(y_test, y_pred_svc))
print("Accuracy score of Logistic Model is :", accuracy_score(y_test, y_pred_log))
print("Accuracy score of Decision Tree is  :", accuracy_score(y_test, y_pred_dt))
print("Accuracy score of Random Forest is  :", accuracy_score(y_test, y_pred_rf))
print("Accuracy score of AdaBoost is       :", accuracy_score(y_test, y_pred_adab))
print("Accuracy score of Gradient Boost is :", accuracy_score(y_test, y_pred_gb))

Accuracy score of KNN Model is      : 0.7142857142857143
Accuracy score of SVC Model is      : 0.6666666666666666
Accuracy score of Logistic Model is : 0.7619047619047619
Accuracy score of Decision Tree is  : 0.9523809523809523
Accuracy score of Random Forest is  : 0.9047619047619048
Accuracy score of AdaBoost is       : 0.9523809523809523
Accuracy score of Gradient Boost is : 0.9523809523809523


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101 entries, 6 to 497
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   age                       101 non-null    int64
 1   annual_income             101 non-null    int64
 2   monthly_spend             101 non-null    int64
 3   avg_session_time          101 non-null    int64
 4   visits_per_month          101 non-null    int64
 5   items_per_purchase        101 non-null    int64
 6   discount_used_pct         101 non-null    int64
 7   days_since_last_purchase  101 non-null    int64
 8   cart_abandon_rate         101 non-null    int64
 9   customer_tenure_months    101 non-null    int64
 10  gender                    101 non-null    int64
 11  region                    101 non-null    int64
 12  device_type               101 non-null    int64
 13  payment_method            101 non-null    int64
 14  membership_type           101 non-null    int64

In [67]:
df.columns

Index(['age', 'annual_income', 'monthly_spend', 'avg_session_time',
       'visits_per_month', 'items_per_purchase', 'discount_used_pct',
       'days_since_last_purchase', 'cart_abandon_rate',
       'customer_tenure_months', 'gender', 'region', 'device_type',
       'payment_method', 'membership_type', 'customer_segment'],
      dtype='object')

In [69]:
data.head()

,age,annual_income,monthly_spend,avg_session_time,visits_per_month,items_per_purchase,discount_used_pct,days_since_last_purchase,cart_abandon_rate,customer_tenure_months,gender,region,device_type,payment_method,membership_type,customer_segment
0,56.0,1.436647e+06,8168.887229,2.018793,10.0,1.0,27.533169,48.0,0.345017,69.0,Male,NaN,Mobile,Credit Card,Platinum,Medium_Value_Customer
1,69.0,1.109740e+06,22375.492576,NaN,15.0,7.0,20.160388,8.0,0.054733,98.0,NaN,West,Mobile,Credit Card,Silver,High_Value_Customer
2,NaN,3.441890e+05,8354.207778,11.683007,NaN,2.0,41.682201,57.0,NaN,106.0,Female,East,Mobile,NaN,Silver,Medium_Value_Customer
3,32.0,6.547298e+05,33426.674454,4.371077,13.0,NaN,40.733550,44.0,0.418695,1.0,Female,NaN,Mobile,UPI,Gold,High_Value_Customer
4,NaN,1.180073e+06,18453.225306,13.017717,NaN,7.0,30.591551,21.0,NaN,39.0,Female,North,Tablet,Net Banking,Platinum,Medium_Value_Customer


In [70]:

unseen_customers = pd.DataFrame([
    {
        'age': 41,
        'annual_income': 850000,
        'monthly_spend': 22000,
        'avg_session_time': 6.8,
        'visits_per_month': 12,
        'items_per_purchase': 4,
        'discount_used_pct': 18,
        'days_since_last_purchase': 10,
        'cart_abandon_rate': 0.25,
        'customer_tenure_months': 36,
        'gender': 'Male',
        'region': 'West',
        'device_type': 'Mobile',
        'payment_method': 'UPI',
        'membership_type': 'Gold'
    },
    {
        'age': 58,
        'annual_income': 1250000,
        'monthly_spend': 14000,
        'avg_session_time': 3.2,
        'visits_per_month': 7,
        'items_per_purchase': 2,
        'discount_used_pct': 8,
        'days_since_last_purchase': 32,
        'cart_abandon_rate': 0.12,
        'customer_tenure_months': 72,
        'gender': 'Female',
        'region': 'North',
        'device_type': 'Tablet',
        'payment_method': 'Credit Card',
        'membership_type': 'Platinum'
    }
])

In [71]:
unseen_customers.head()

,age,annual_income,monthly_spend,avg_session_time,visits_per_month,items_per_purchase,discount_used_pct,days_since_last_purchase,cart_abandon_rate,customer_tenure_months,gender,region,device_type,payment_method,membership_type
0,41,850000,22000,6.8,12,4,18,10,0.25,36,Male,West,Mobile,UPI,Gold
1,58,1250000,14000,3.2,7,2,8,32,0.12,72,Female,North,Tablet,Credit Card,Platinum
